# **Demos of MultiResUNet models implemented on the CelebAMaskHQ dataset**
In this notebook, we display demos of the models tested using the mechanisms mentioned in [MultiResUNet.ipynb](https://drive.google.com/file/d/1H26uaN10rU2V7MnX8vRdE3J0ZMoO7wq2/view?usp=sharing).
This demo should work irrespective of any access issues to the author's drive.
If any errors in downloading models may occur, kindly notify me.

We use gdown to install the JSON files (per model) and their weights for testing, and user input image is utilised in order to provide output.

Note : Due to Colab's anomalous nature with matplotlib.pyplot, we will store the final images within the machine.

PS : If you would like to see the results for all the demo networks, utilise the 'Run All' feature in the Runtime submenu.

Run the snippet below in order to download all the models and their corresponding weights:

You can download the models by going through the links mentioned in the comments below-

In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
!pip install gdown

# The testing model -
# https://drive.google.com/file/d/1-0H74nlLkTnnvSkMG-MPuhuX70LXBOlo/view?usp=sharing
# https://drive.google.com/file/d/1--fVnHrfDpujmdX9OWT6PVmNQjQ1mPX8/view?usp=sharing

# The F10k model -
# https://drive.google.com/file/d/1-GhqkzttGHAkGi0r5XEgowVg8wLcgmY1/view?usp=sharing
# https://drive.google.com/file/d/1-CPthO3qPHE_IeykqyG_bndf9UuQbHAO/view?usp=sharing

# The FD10 model -
# https://drive.google.com/file/d/1yhWML6lThGv_MSGkUOuVhkoD7H-u6gwm/view?usp=sharing
# https://drive.google.com/file/d/12S277zHGFN9YPKcX7M7hEwxkokvT5bNt/view?usp=sharing

# For the test model :
!gdown --id 1-0H74nlLkTnnvSkMG-MPuhuX70LXBOlo --output modelP5.json
!gdown --id 1--fVnHrfDpujmdX9OWT6PVmNQjQ1mPX8 --output modelW.h5

# For the F10k model :
!gdown --id 1-GhqkzttGHAkGi0r5XEgowVg8wLcgmY1 --output modelP5f10.json
!gdown --id 1-CPthO3qPHE_IeykqyG_bndf9UuQbHAO --output modelWf10.h5

# For the FD10 model :
!gdown --id 1yhWML6lThGv_MSGkUOuVhkoD7H-u6gwm --output modelP5FD.json
!gdown --id 12S277zHGFN9YPKcX7M7hEwxkokvT5bNt --output modelWFD.h5


!ls


Using TensorFlow backend.


Downloading...
From: https://drive.google.com/uc?id=1-0H74nlLkTnnvSkMG-MPuhuX70LXBOlo
To: /content/modelP5.json
100% 117k/117k [00:00<00:00, 35.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--fVnHrfDpujmdX9OWT6PVmNQjQ1mPX8
To: /content/modelW.h5
29.6MB [00:00, 57.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GhqkzttGHAkGi0r5XEgowVg8wLcgmY1
To: /content/modelP5f10.json
100% 117k/117k [00:00<00:00, 42.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-CPthO3qPHE_IeykqyG_bndf9UuQbHAO
To: /content/modelWf10.h5
29.6MB [00:00, 138MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1yhWML6lThGv_MSGkUOuVhkoD7H-u6gwm
To: /content/modelP5FD.json
100% 117k/117k [00:00<00:00, 35.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=12S277zHGFN9YPKcX7M7hEwxkokvT5bNt
To: /content/modelWFD.h5
29.6MB [00:00, 181MB/s]
15000.jpg	 modelP5FD.json  modelWFD.h5	     results_modelP5FD
29987.jpg	 modelP5.json	 modelW.h5	     sample_data
modelP5f10.json  model

### Image input and pre processing :
Run the cell below in order to 
upload the required images to be tested.
Accepts .jpg format images.

In [0]:
from google.colab import files

img_arr = []
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    
    if(fn.split('.')[-1]=='jpg'or fn.split('.')[-1]=='jpeg'):
        img = cv2.imread(fn, cv2.IMREAD_COLOR)
        img_arr.append(cv2.resize(img,(256, 192), interpolation = cv2.INTER_CUBIC))
    else:
        print(fn+' is not of the valid format.')
img_loaded = img_arr
img_arr = np.array(img_arr)        
img_arr = img_arr / 255
print('Number of images uploaded : '+str(len(img_arr)))
img_names = list(uploaded.keys())

Saving 29987.jpg to 29987 (1).jpg
Saving 29978.jpg to 29978.jpg
User uploaded file "29987.jpg" with length 55367 bytes
User uploaded file "29978.jpg" with length 65300 bytes
Number of images uploaded : 2


## Boilerplate code to run model :
This code provides outputs in the format(image-mask-imagewithmask).

Your results will be stored under the Files Section (on the left side of the website) in the folder specified by the output during runtime.

In order to allow for automatic downloading of the images, just uncomment the
```
#files.download('results_'+model_json.split('.')[0]+'/result_'+str(img_names[i].split('.')[0])+'.png')
```
section of the code below. 
(NOTE : This feature works only for **Google Chrome** users)



In [0]:
from google.colab import files
from keras.models import model_from_json

def RunModel( model_json, model_weights, image_array, img_names, img_loaded):

    try:
        os.makedirs('results_'+model_json.split('.')[0])
    except:
        pass
    
    print('Your results will be stored under the Files Section in the folder '+'results_'+model_json.split('.')[0])
    # load json and create model
    json_file = open(model_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(model_weights)
    print("Loaded model from disk")
    loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Used in order to ease naming
    i = 0
    yp = loaded_model.predict(x=np.array(image_array), batch_size=1, verbose=1)
    yp = np.round(yp,0)
    
    for ip_img in image_array:
        
        # Modification of mask in order to mimic sample output 
        t = yp[i]
        a = np.concatenate((t,t),axis = 2)
        b = np.concatenate((a,t),axis = 2)
        b = b * 255
        
        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.imshow(cv2.cvtColor(np.array(img_loaded[i]), cv2.COLOR_BGR2RGB))
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        plt.title('Prediction')
        plt.subplot(1,3,3)
        plt.imshow(cv2.cvtColor(cv2.addWeighted(np.uint8(img_loaded[i]), 0.5, np.uint8(b), 0.5, 0.0),cv2.COLOR_BGR2RGB))
        plt.title('Prediction with mask')
        plt.savefig('results_'+model_json.split('.')[0]+'/result_'+str(img_names[i].split('.')[0])+'.png',format='png')
        plt.close()
        # Uncomment the line below to allow automatic downloading
        #files.download('results_'+model_json.split('.')[0]+'/result_'+str(img_names[i].split('.')[0])+'.png')
        i += 1


# **Model F10k:**
This model has been trained on a 80%-20% split amongst the first 10,000 images of the dataset.

*   Number of epochs: 20
*   Time taken : [1:29:40, 267.74s/it]
*   Jacard Index(final) : 0.8452393049122288
*   Dice Coefficient(final) : 0.9139967587791317
*   Accuracy(final) : 99.80 %

In [0]:
RunModel('modelP5f10.json','modelWf10.h5', img_arr, img_names, img_loaded)

Your results will be stored under the Files Section in the folder results_modelP5f10
Loaded model from disk
2/2 [==============================] - 5s 3s/step


# **Model FD10:**

The dataset for this model has been split twice: 

1. Three sets of 10,000 images each.
2. Each set trained on a 80%-20% split.

Each split of the data was used to train the model for 10 epochs each. The split was performed in order to compensate for RAM bottlenecks in our system. 

*   Number of epochs: 10+10+10 = 30
*   Time taken : [2:29:04, 2958.94s/it]
*   Jacard Index(final) : 0.8331437322988224
*   Dice Coefficient(final) : 0.9071035040844939
*   Accuracy(final) : 99.70 %

In [0]:
RunModel('modelP5FD.json','modelWFD.h5',img_arr,img_names,img_loaded)

Your results will be stored under the Files Section in the folder results_modelP5FD












Loaded model from disk


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2/2 [==============================] - 4s 2s/step


# **EXTRA : Base Testing Model**

**This model returns only the left eye segmented as a mask and was made for TESTING PURPOSES ONLY**

This model has been trained on a 80%-20% split amongst the first 250 images of the dataset, and was used in order to test the original model mentioned in the paper.

*   Number of epochs: 10
*   Time taken : [0:20:40]
*   Jacard Index(final) : 0.39899180087352404
*   Dice Coefficient(final) : 0.495551362130639337
*   Accuracy(final) : 99.80 %

**NOTE : THIS MODEL IS SIMPLY A PRECURSOR TO OUR ACTUAL MODELS MENTIONED ABOVE AND SHOULD NOT BE CONSIDERED AS FEASIBLE FOR ANY ASPECTS**

In [0]:
RunModel('modelP5.json','modelW.h5',img_arr,img_names,img_loaded)